In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
df1 = pd.read_csv('wine1.csv')
df2 = pd.read_csv('wine2.csv')


# testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
# print(testimonial.sentiment)

Populating the interactive namespace from numpy and matplotlib


In [72]:
frames = [df1,df2]

df = pd.concat(frames, sort = True)
df["province country"] = df["province"] + ", " + df["country"]



In [64]:
def sentiment(description):
    des = TextBlob(description)
    return des.sentiment.polarity


In [65]:
#best/worst average by country


avg_country = df[["country","points"]].groupby("country").agg(["count","mean"])
avg_country.columns = ['Count', "Mean"]
avg_country = avg_country[avg_country["Count"] > 200]
avg_country.sort_values(by= "Mean", ascending = False)[0:5] #best
#avg_country.sort_values(by= "Mean", ascending = True)[0:5] #worst

,Count,Mean
country,,
Austria,6402,89.707591
Germany,4617,89.200996
France,43191,88.884559
Canada,453,88.880795
Italy,43018,88.481147


In [66]:
#best average by region
avg_region = df[["province country","points"]].groupby(["province country"]).agg(["count","mean"])

avg_region.columns = ['Count', "Mean"]
avg_region = avg_region[avg_region["Count"] > 100]

avg_region.sort_values(by="Mean", ascending = False)[0:5] #best

avg_region.sort_values(by="Mean", ascending = True)[0:5] #worst



,Count,Mean
province country,,
"New Mexico, US",140,83.950000
"Central Valley, Chile",884,84.830317
"Central Spain, Spain",1228,84.906352
"Valle de Guadalupe, Mexico",122,84.909836
"Australia Other, Australia",798,85.030075


In [67]:
#best average by varietal
avg_varietal = df[["variety","points"]].groupby(["variety"]).agg(["count",'mean'])

avg_varietal.columns = ['Count', "Mean"]
avg_varietal = avg_varietal[avg_varietal["Count"] > 100]


avg_varietal.sort_values(by="Mean", ascending = False)[0:5] #best
avg_varietal.sort_values(by="Mean", ascending = True)[0:5] #worst



,Count,Mean
variety,,
Lambrusco,110,84.409091
Viura,217,85.092166
Torrontés,617,85.202593
Chardonnay-Viognier,110,85.618182
Garnacha,727,85.656121


In [68]:
#best/worst bang per buck per country
df["BPB Score"] = df["points"]/df["price"]

bbp_country = df[["country","BPB Score"]].groupby("country").agg(["count","mean"])
bbp_country.columns = ['Count', "Mean"]

bbp_country2 = bbp_country[bbp_country["Count"] > 200]
bbp_country2.sort_values(by= "Mean", ascending = False)[0:5] #best
#bbp_country2.sort_values(by= "Mean", ascending = True)[0:5] #worst

,Count,Mean
country,,
Romania,259,8.249066
Bulgaria,218,7.488149
Chile,10182,5.938419
Argentina,9343,5.611222
Portugal,9051,5.554613


In [69]:
pd.qcut(df["price"], 15)

0         (75.0, 3300.0]
1         (75.0, 3300.0]
2         (75.0, 3300.0]
3           (55.0, 75.0]
4           (55.0, 75.0]
5           (55.0, 75.0]
6           (55.0, 75.0]
7         (75.0, 3300.0]
8           (55.0, 75.0]
9           (55.0, 75.0]
10        (75.0, 3300.0]
11          (45.0, 55.0]
12          (45.0, 55.0]
13        (75.0, 3300.0]
14        (75.0, 3300.0]
15        (75.0, 3300.0]
16        (75.0, 3300.0]
17        (75.0, 3300.0]
18        (75.0, 3300.0]
19          (55.0, 75.0]
20          (20.0, 24.0]
21        (75.0, 3300.0]
22        (75.0, 3300.0]
23          (55.0, 75.0]
24          (40.0, 45.0]
25          (55.0, 75.0]
26          (55.0, 75.0]
27        (75.0, 3300.0]
28          (55.0, 75.0]
29          (55.0, 75.0]
               ...      
129941      (19.0, 20.0]
129942      (30.0, 35.0]
129943      (26.0, 30.0]
129944      (24.0, 26.0]
129945      (19.0, 20.0]
129946      (15.0, 17.0]
129947      (19.0, 20.0]
129948      (40.0, 45.0]
129949      (30.0, 35.0]
